In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 17 09:59:27 2021

@author: mcalderonloor
"""
import ee
ee.Initialize()
import numpy as np
import matplotlib.pyplot as plt
from numpy import asarray
from numpy import savetxt
import pandas as pd
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to csv file


#from osgeo import gdal
#from osgeo import osr
#import time

#crop = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_anuclim_nneigh_ndist_naccess_tenure_cropland').rename('prob_crop')
#forest = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_anuclim_nneigh_ndist_naccess_tenure_forest').rename('prob_forest')
#grass = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_anuclim_nneigh_ndist_naccess_tenure_grassland').rename('prob_grass')
#urb = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_anuclim_nneigh_ndist_naccess_tenure_urban').rename('prob_urban')
#other = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_anuclim_nneigh_ndist_naccess_tenure_other').rename('prob_other')


crop = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_tunnedv1_cropland').rename('prob_crop')
forest = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_tunnedv1_forest').rename('prob_forest')
grass = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_tunnedv1_grassland').rename('prob_grass')
urb = ee.Image('users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_tunnedv1_urban').rename('prob_urban')

urb1= ee.Image("users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_tunnedv1_n67_n27_urban").rename('prob_urban39')
urb2= ee.Image("users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_tunnedv1_n67_urban").rename('prob_urban3927')
urb3= ee.Image("users/mioash/Calderon_etal_Australian_land-cover/Tas_lc2010_tunnedv1_n67_n27_n9urban").rename('prob_urban3')


lc_aus = ee.Image("users/mioash/Calderon_etal_Australian_land-cover/Lc_Aus_1985_2015_v1")
sgrid = ee.FeatureCollection("users/mioash/drivers_lcaus/boundaries/tasmania_grid_v2302")

probs = crop.addBands(forest).addBands(grass).addBands(urb).addBands(urb1).addBands(urb2).addBands(urb3)


#probs= probs.multiply(10000000000000000)
probs = probs.addBands(lc_aus.select(['b2010','b2005']))
#urb = urb.multiply(10000000000000000)

# tas_sa2 = ee.FeatureCollection("users/mioash/transitions/tas_sa2_pixels")
# tas_sa2_summary = ee.FeatureCollection("users/mioash/transitions/tas_summary_sa2")
sa2 = ee.FeatureCollection("users/mioash/drivers_lcaus/boundaries/SA2")
sa2_tas = sa2.filterMetadata('STE_CODE11','equals','6')
id_look = 'SA2_MAIN11'
#sa2_id = [601021010, 601021005]  #601021011
# sa2_id = 601021010 #603011065 good accuracy

def id_parse (FCC):
     return ee.Feature(FCC).set(id_look, ee.Number.parse(FCC.get(id_look)))

sa2_tas = sa2_tas.map(id_parse)


#tas_hex = ee.FeatureCollection('users/mioash/tas_hex_clipped2')
#th = tas_hex.filterMetadata('Id_hex_new','equals',13)

sa2_painted = ee.Image(-9999).paint(sa2_tas,id_look).rename('sa2_id')
sa2_painted = sa2_painted.clipToCollection(sgrid)

#sa_test = sa2_tas.filterMetadata(id_look,'equals',sa2_id)#filter(ee.Filter.inList(id_look, sa2_id))#.filterMetadata(id_look,'equals',sa2_id)

probs = probs.unmask(-9999,False).addBands(sa2_painted)

#probs = probs#.toInt8()

In [38]:
def LatLonImg(img,region):
   
    img = img.addBands(ee.Image.pixelLonLat())
    
    img = img.reduceRegion(reducer=ee.Reducer.toList(),\
                                        geometry=region,\
                                        maxPixels=1e13,\
                                        scale=30,\
                                        tileScale= 2);
 
    sa2id= np.array((ee.Array(img.get("sa2_id")).getInfo()))#.astype('i4')
    if np.max(sa2id) != -9999:        
        lats = np.array((ee.Array(img.get("latitude")).getInfo()))#.astype('i4')
        lons = np.array((ee.Array(img.get("longitude")).getInfo()))#.astype('i4')
        cr = np.array((ee.Array(img.get("prob_crop")).getInfo()))#.astype('i4')
        fr = np.array((ee.Array(img.get("prob_forest")).getInfo()))#.astype('i4')
        gr= np.array((ee.Array(img.get("prob_grass")).getInfo()))#.astype('i4')
        ur= np.array((ee.Array(img.get("prob_urban")).getInfo()))#.astype('i4')
        ur1= np.array((ee.Array(img.get("prob_urban39")).getInfo()))#.astype('i4')
        ur2= np.array((ee.Array(img.get("prob_urban3927")).getInfo()))#.astype('i4')
        ur3= np.array((ee.Array(img.get("prob_urban3")).getInfo()))#.astype('i4')
        lprev= np.array((ee.Array(img.get("b2005")).getInfo()))#.astype('i4')
        lpred= np.array((ee.Array(img.get("b2010")).getInfo()))#.astype('i4')
        
        arr = np.vstack([lats, lons, sa2id, cr, fr, gr, ur, ur1, ur2 , ur3 ,lprev, lpred]).transpose()
        return arr[arr[:,2] != -9999,:]
     #lol[lol[:,9] == 13,:]
        
    # try:
    #     data = np.array((ee.Array(img.get("classification")).getInfo()))
    # except:
    #     data= np.full_like(lats, np.nan,dtype=np.float32)
    
    #return lats, lons, cr, fr, gr, ur, ot, lprev, lpred

In [29]:
import geemap
Map = geemap.Map(center=[-37.74,144.93], zoom=7)

Map.add_basemap("SATELLITE")

Map.addLayer(probs.select('prob_crop'), {'min':0,'max':1,'palette':['red','orange','blue']}, 'Prob_crop')
Map.addLayer(probs, '', 'Probs')
Map.addLayer(sgrid, '', 'sgrid')
Map

Map(center=[-37.74, 144.93], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(…

In [48]:
llist = list(range(0,289))
#llist = [60,61]
#sgrid1 = sgrid.filterMetadata('id_grid','equals',143) #143 yes, 163:water
#print(llist[3])

appended = np.zeros([1,12])

for i in range(200,289,1):
    sgrid1 = sgrid.filterMetadata('id_grid', 'equals',llist[i])
    probs1 = probs.clip(sgrid1)
    limg = LatLonImg(probs1,sgrid1)
    #appended = np.concatenate((appended,limg),0)
    if limg is None:
        print('Bad:',i)
    else:
        appended = np.concatenate((appended,limg),0)

np.save('tas_200_289_v1',appended)
b1 = probs.bandNames().remove(str('sa2_id')).getInfo()
b1 = ['latitude','longitude','sa2_id']+ b1
print(b1)
#print(appended[1:10,])
print(np.shape(appended))
pts_pd = pd.DataFrame(data=appended, columns=b1)
pts_pd.head()

Bad: 200
Bad: 201
Bad: 202
Bad: 215
Bad: 216
Bad: 217
Bad: 218
Bad: 220
Bad: 221
Bad: 232
Bad: 233
Bad: 234
Bad: 235
Bad: 236
Bad: 237
Bad: 238
Bad: 249
Bad: 253
Bad: 254
Bad: 255
Bad: 270
Bad: 271
Bad: 272
Bad: 273
Bad: 274
Bad: 275
Bad: 276
Bad: 286
Bad: 287
Bad: 288
['latitude', 'longitude', 'sa2_id', 'prob_crop', 'prob_forest', 'prob_grass', 'prob_urban', 'prob_urban39', 'prob_urban3927', 'prob_urban3', 'b2010', 'b2005']
(34340796, 12)


,latitude,longitude,sa2_id,prob_crop,prob_forest,prob_grass,prob_urban,prob_urban39,prob_urban3927,prob_urban3,b2010,b2005
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-39.203782,147.003233,602031063.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,-39.204051,147.002694,602031063.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,-39.204051,147.002963,602031063.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,-39.204051,147.003233,602031063.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [42]:
np.save('tas_0_99_v1',appended)
#np.unique(appended[:,2])
#np.save('tas_200_289',appended)

In [47]:
llist[288]

288